# check oof df

In [1]:
import os
import gc
import yaml
import argparse

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch

from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append("/kaggle/src/dss_utils")
sys.path.append("/kaggle/src/exp")
sys.path.append("/kaggle/src/model")
sys.path.append("/kaggle/src/data")
from dss_metrics import score
from training_loop import get_valid_values_dict, concat_valid_input_info, get_oof_df, get_key_df
from dss_model import get_model
from dss_dataloader import get_loader


In [2]:
output_dir = "/kaggle/working"


In [3]:
# exp_name = "exp1st000_baseline_epoch1_1epoch"
# exp_name = "exp1st000_baseline_epoch10"
detect_exp_name = "exp1st000_baseline_convkernel192_epoch10"
folds = [0, 1, 2, 3, 4]
# folds = [0]
oof_detect_target_df = pd.DataFrame()
for i in folds:
    print("fold", i)
    df = pd.read_parquet(os.path.join(output_dir, "_oof", detect_exp_name, f"fold{i}_best_pred_df.parquet"))
    # df = pd.read_csv(os.path.join(output_dir, "_oof", exp_name, f"fold{i}_best_pred_df.parquet"))
    oof_detect_target_df = pd.concat([oof_detect_target_df, df], axis=0)


fold 0
fold 1
fold 2
fold 3
fold 4


In [4]:
check_target_df = oof_detect_target_df[["class_preds", "class_targets"]]
for thr in [0.01, 0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.14, 0.16, 0.18, 0.20]:
    preds = (check_target_df["class_preds"].values >= thr).astype(int)
    conf_mat = confusion_matrix(check_target_df["class_targets"].values, preds)
    tn, fp, fn, tp = conf_mat.ravel()
    print("thr=", thr, "tn", tn,"fp", fp, "fn", fn, "tp", tp)


thr= 0.01 tn 0 fp 2931 fn 0 tp 4815
thr= 0.02 tn 2 fp 2929 fn 0 tp 4815
thr= 0.04 tn 201 fp 2730 fn 0 tp 4815
thr= 0.06 tn 628 fp 2303 fn 0 tp 4815
thr= 0.08 tn 782 fp 2149 fn 2 tp 4813
thr= 0.1 tn 883 fp 2048 fn 4 tp 4811
thr= 0.12 tn 958 fp 1973 fn 4 tp 4811
thr= 0.14 tn 1200 fp 1731 fn 7 tp 4808
thr= 0.16 tn 1649 fp 1282 fn 11 tp 4804
thr= 0.18 tn 1919 fp 1012 fn 14 tp 4801
thr= 0.2 tn 2024 fp 907 fn 17 tp 4798


exp1st000_baseline_epoch10  
thr= 0.01 tn 1881 fp 1050 fn 9 tp 4806  
thr= 0.02 tn 2056 fp 875 fn 12 tp 4803  
thr= 0.04 tn 2199 fp 732 fn 18 tp 4797  
thr= 0.06 tn 2299 fp 632 fn 19 tp 4796  
thr= 0.08 tn 2338 fp 593 fn 25 tp 4790  
thr= 0.1 tn 2379 fp 552 fn 31 tp 4784  

In [5]:
exp_name = "exp025_alldata_skf_lstmenchead_epoch30"
folds = [0, 1, 2, 3, 4]
# folds = [0]
oof_df = pd.DataFrame()
for i in folds:
    print("fold", i)
    # df = pd.read_parquet(os.path.join(output_dir, "_oof", exp_name, f"raw_oof_df_fold{i}.parquet"))
    df = pd.read_parquet(os.path.join(output_dir, "_oof", exp_name, f"fold{i}_best_oof_df.parquet"))
    oof_df = pd.concat([oof_df, df], axis=0)


print(len(oof_df["series_id"].unique()))   


fold 0
fold 1
fold 2
fold 3
fold 4
277


In [6]:
oof_detect_target_df


,series_date_key,start_step,end_step,class_preds,class_targets
0,24,0,5759,0.092254,0.0
1,25,5760,23039,0.912998,1.0
2,26,23040,40319,0.922159,1.0
3,27,40320,57599,0.917359,1.0
4,28,57600,74879,0.088085,0.0
...,...,...,...,...,...
1500,7300,680580,697859,0.157540,0.0
1501,7301,697860,715139,0.157540,0.0
1502,7302,715140,732419,0.157540,0.0
1503,7303,732420,749699,0.157540,0.0


In [7]:
oof_detect_target_df = pd.merge(oof_detect_target_df, oof_df[["series_date_key", "series_id"]], on="series_date_key", how="left")
oof_detect_target_df = oof_detect_target_df.drop_duplicates(subset=["series_date_key", "series_id"])


In [8]:
oof_detect_target_df


,series_date_key,start_step,end_step,class_preds,class_targets,series_id
0,24,0,5759,0.092254,0.0,03d92c9f6f8a
5760,25,5760,23039,0.912998,1.0,03d92c9f6f8a
23040,26,23040,40319,0.922159,1.0,03d92c9f6f8a
40320,27,40320,57599,0.917359,1.0,03d92c9f6f8a
57600,28,57600,74879,0.088085,0.0,03d92c9f6f8a
...,...,...,...,...,...,...
127869549,7300,680580,697859,0.157540,0.0,f0482490923c
127886829,7301,697860,715139,0.157540,0.0,f0482490923c
127904109,7302,715140,732419,0.157540,0.0,f0482490923c
127921389,7303,732420,749699,0.157540,0.0,f0482490923c


In [9]:
oof_detect_target_df = oof_detect_target_df[["series_id", "start_step", "end_step", "class_preds", "class_targets"]]
oof_detect_target_df = oof_detect_target_df.rename(columns={"class_preds": "preds", "class_targets": "targets"})


In [12]:
overall_oof_path = os.path.join(output_dir, "_overall_oof", detect_exp_name)
os.makedirs(overall_oof_path, exist_ok=True)
oof_detect_target_df.to_parquet(os.path.join(overall_oof_path, "oof_detect_df.parquet"), index=False)


In [7]:
oof_df = oof_df.sort_values(["series_id", "step"])
oof_df = oof_df[["series_id", "series_date_key",  "step", "second", "event_pred", "class_pred"]]


In [16]:
# postprocess_fn
# series_idでgroupbyして、class_predに対して対象の列のデータから前のN個の列までのデータの平均をとる
import torch
import torch.nn as nn

def detect_event_from_classpred(df, detect_taret_df, N=300, maxpool_kernel_size=41, detect_thr=0.01):
    not_detect_target_keys = detect_taret_df[detect_taret_df["class_preds"] < detect_thr]["series_date_key"].values.tolist()
    df = df.copy()

    # series_idでgroupbyして、class_predに対して対象の列のデータから
    # 前のN個の列までのデータの平均をとる
    df["class_pred_beforemean"] = df.groupby("series_id")["class_pred"].apply(
        lambda x: x.rolling(N, min_periods=1).mean()
    )
    df["class_pred_aftermean"] = df.groupby("series_id")["class_pred"].apply(
        lambda x: x[::-1].rolling(N, min_periods=1).mean()[::-1]
    )
    df["event_pred"] = df["class_pred_beforemean"] - df["class_pred_aftermean"]
    # df.loc[df["series_date_key"].isin(not_detect_target_keys), "event_pred"] = 0
    # 入力サイズと出力サイズが一致するようにpaddingを調整
    maxpool_padding = int((maxpool_kernel_size - 1) / 2)
    # maxpoolしてピーク検出
    max_pooling = nn.MaxPool1d(
        maxpool_kernel_size, stride=1, padding=maxpool_padding
    )
    event_pred = df["event_pred"].values
    event_pred = torch.tensor(event_pred).unsqueeze(0)
    pooled_event_pred = max_pooling(np.abs(event_pred)).squeeze(0).numpy()
    event_pred = event_pred.squeeze(0).numpy()
    # peakのところだけ残すmaskを作成
    peak_event_pred_mask = np.where(pooled_event_pred == np.abs(event_pred), 1, 0)
    peak_event_pred = event_pred * peak_event_pred_mask
    df["event_pred"] = peak_event_pred
    df.loc[df["series_date_key"].isin(not_detect_target_keys), "event_pred"] = 0

    # detect_taret_df = detect_taret_df.rename(columns={"class_preds": "detect_pred"})
    # detect_taret_df["detect_pred"] = np.where(detect_taret_df["detect_pred"] > detect_thr, 1, detect_taret_df["detect_pred"])
    # df = pd.merge(df, detect_taret_df[["series_date_key", "detect_pred"]], on="series_date_key", how="left")
    # df["event_pred"] = df["event_pred"] * df["detect_pred"]
    df["onset"] = np.clip(-df["event_pred"], 0, 1)
    df["wakeup"] = np.clip(df["event_pred"], 0, 1)

    df = df.drop(["class_pred_beforemean", "class_pred_aftermean"], axis=1)

    return df

def make_submission_df(df, threshold=0.1):
    df = df[["series_id", "step", "event_pred"]].copy()
    # thresholdより大きいときは1,-thresholdより小さいときは-1,それ以外は0
    df["event"] = df["event_pred"].apply(lambda x: 1 if x > threshold else -1 if x < -threshold else 0)
    df = df[df["event"] != 0].copy()
    df["event"] = df["event"].replace({1: "wakeup", -1: "onset"})
    df["score"] = df["event_pred"].apply(lambda x: np.clip(np.abs(x), 0.0, 1.0))
    return df


In [11]:
train_event_df = pd.read_csv("/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv")
train_event_df = train_event_df[train_event_df["step"].notnull()].copy()
# train_event_df = train_event_df[train_event_df["series_id"].isin(oof_df["series_id"].unique())].copy()
train_event_df = train_event_df.reset_index(drop=True)
display(train_event_df.head())


,series_id,night,event,step,timestamp
0,038441c925bb,1,onset,4992.0,2018-08-14T22:26:00-0400
1,038441c925bb,1,wakeup,10932.0,2018-08-15T06:41:00-0400
2,038441c925bb,2,onset,20244.0,2018-08-15T19:37:00-0400
3,038441c925bb,2,wakeup,27492.0,2018-08-16T05:41:00-0400
4,038441c925bb,3,onset,39996.0,2018-08-16T23:03:00-0400


In [17]:
average_size = 132
max_kernel_size_ = 33
# detect_thr = 0.01
detect_thr = 0.2
df = detect_event_from_classpred(oof_df, oof_detect_target_df, N=average_size, maxpool_kernel_size=max_kernel_size_, detect_thr=detect_thr)
sub_df = make_submission_df(df, threshold=0.01)
print(average_size, max_kernel_size_)

if len(sub_df) == 0:
    print("event not detected")
    score_ = 0
else:
    score_ = score(train_event_df, sub_df)
print(score_)


132 33
0.7499010427377746


1 or pred value
0.7501632817105435

In [13]:
average_size = 132
max_kernel_size_ = 33
# detect_thr = 0.01
# detect_thr = 0.2

print(average_size, max_kernel_size_)
for detect_thr in [0.1, 0.15, 0.2, 0.25, 0.3]:
    print(detect_thr)
    df = detect_event_from_classpred(oof_df, oof_detect_target_df, N=average_size, maxpool_kernel_size=max_kernel_size_, detect_thr=detect_thr)
    sub_df = make_submission_df(df, threshold=0.01)

    if len(sub_df) == 0:
        print("event not detected")
        score_ = 0
    else:
        score_ = score(train_event_df, sub_df)
    print(score_)


132 33
0.25
0.7500886278505996
0.3
0.750159416120361


1 or pred value
132 33  
0.25 0.7500886278505996  
0.3 0.750159416120361  

exp025_alldata_skf_lstmenchead_epoch30
132 33
0.7500099409982081